## 3D CNN for MRI 

### Importing packages


In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.python.framework import ops

%matplotlib inline

np.random.seed(1)

/home/root_zero/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def placeholders(shape,n_Ch,n_Y):
   
    n_D,n_R,n_C=shape
    X = tf.placeholder(shape=(None, n_D,n_R,n_C,n_Ch),dtype='float',name="X")  #input placeholder 
    Y= tf.placeholder(shape=(None,n_Y),dtype="float",name="Y")   #output placeholder
    
    return X,Y
    
    

In [3]:
shape= [30,64,64]
X,Y= placeholders(shape,3,1)
print ("X = " + str(X))
print ("Y = " + str(Y))

X = Tensor("X:0", shape=(?, 30, 64, 64, 3), dtype=float32)
Y = Tensor("Y:0", shape=(?, 1), dtype=float32)


In [4]:
def initialize_parameters(n_Ch):
    
    tf.set_random_seed(1)

    W1 = tf.get_variable("W1",[5,5,5,n_Ch,8],initializer=tf.contrib.layers.xavier_initializer(seed = 2)) #weight for hidden layer1
    W2 = tf.get_variable("W2",[3,3,3,8,20],initializer=tf.contrib.layers.xavier_initializer(seed = 2)) #weight for hidden layer2
    W3 = tf.get_variable("W3",[2,2,2,20,30],initializer=tf.contrib.layers.xavier_initializer(seed = 2))#weight for hidden layer3
    
    parameters = {"W1": W1,
                  "W2": W2,
                 "W3":W3}
    return parameters

In [5]:
tf.reset_default_graph()
with tf.Session() as sess_test:
    parameters = initialize_parameters(3)
    init = tf.global_variables_initializer()
    sess_test.run(init)
    print("W1 = " + str(parameters["W1"].eval()[1,1,1]))
    print("W2 = " + str(parameters["W2"].eval()[1,1,1]))

W1 = [[-0.03683396 -0.01747156  0.0626035  -0.0396937   0.06305829  0.01175921
  -0.03583478 -0.04021666]
 [-0.02612016 -0.04518278  0.02921299  0.00691286  0.01202715  0.02822192
   0.01201055 -0.05580938]
 [ 0.01291112 -0.04221325 -0.04847242  0.03618843  0.01366609 -0.05239179
  -0.04156421  0.04751361]]
W2 = [[ 0.02904094 -0.01707939 -0.01908917  0.02339057 -0.01865383  0.07799385
   0.02017494 -0.07534269  0.08736709  0.0246733   0.01382897 -0.05351781
  -0.06159095 -0.05348472 -0.01091406 -0.0707201  -0.03715204 -0.0560387
   0.01583225  0.08135979]
 [ 0.07302515  0.02072123 -0.02753112  0.05069304 -0.06763384 -0.04402454
   0.02812131 -0.02235015  0.03556526  0.04793748  0.07305662  0.00805596
  -0.06419431 -0.05185527 -0.08084551  0.01047558  0.05220492  0.03698102
   0.04131581 -0.05287965]
 [-0.05757724  0.00040872 -0.01496833 -0.01075824 -0.03189651  0.05547701
  -0.08840938  0.06412901 -0.05141369  0.06284249  0.06560351  0.04337619
   0.01627102  0.05671664 -0.0011998  -0.

## Forward Prop


In [7]:
def forward_prop(X,parameters):
    
    W1 = parameters['W1']
    W2 = parameters['W2']
    W3 = parameters['W3']
    
    
    _,n_D,n_R,n_C,_=X.shape
    #Convolution layer
    Z1 = tf.nn.conv3d(X,W1, strides = [1,1,1,1,1], padding = 'SAME')
    #Relu
    A1=tf.nn.relu(Z1)
    #Pooling Layer
    P1 = tf.nn.max_pool3d(A1, ksize = [1,8,8,8,1], strides = [1,8,8,8,1], padding = 'SAME')
    #Convolution layer 2
    Z2 = tf.nn.conv3d(P1,W2, strides = [1,1,1,1,1], padding = 'SAME')
    #Relu 2
    A2=tf.nn.relu(Z2)
    #Pooling Layer2
    P2 = tf.nn.max_pool3d(A2, ksize = [1,5,5,5,1], strides = [1,5,5,5,1], padding = 'SAME')
    #Convolution layer 3
    Z3 = tf.nn.conv3d(P2,W3, strides = [1,1,1,1,1], padding = 'SAME')
    #Relu 3
    A3=tf.nn.relu(Z3)
    #Pooling Layer 3
    P3 = tf.nn.max_pool3d(A3, ksize = [1,2,2,2,1], strides = [1,2,2,2,1], padding = 'SAME')
    #fullyconnected layer
    Z3=tf.contrib.layers.fully_connected(P3,1,activation_fn=None)
    
    return Z3

In [8]:
tf.reset_default_graph()

with tf.Session() as sess:
    np.random.seed(1)
   
    X, Y = placeholders(shape,3,1)
    parameters = initialize_parameters(3)
    Z3 = forward_prop(X, parameters)
    init = tf.global_variables_initializer()
    sess.run(init)
    a = sess.run(Z3, {X: np.random.randn(2,30,64,64,3), Y: np.random.randn(2,1)})
    print("Z3 = " + str(a))

Z3 = [[[[[0.8273127]]]]



 [[[[0.8643329]]]]]


In [9]:
def com_cost(Z3, Y):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = Z3, labels = Y)) #cost function to calculate loss at each time step
    return cost

In [10]:
tf.reset_default_graph()

with tf.Session() as sess:
    np.random.seed(1)
   
    X, Y = placeholders(shape,3,1)
    parameters = initialize_parameters(3)
    Z3 = forward_prop(X, parameters)
    init = tf.global_variables_initializer()
    sess.run(init)
    a = sess.run(Z3, {X: np.random.randn(2,30,64,64,3), Y: np.random.randn(2,1)})
    print("cost = " + str(a))

cost = [[[[[0.8273127]]]]



 [[[[0.8643329]]]]]


In [11]:
def random_mini_batches(X, Y, mini_batch_size = 64, seed = 0):

    
    m = X.shape[0]                  
    mini_batches = []
    np.random.seed(seed)
    
 
    permutation = list(np.random.permutation(m))
    shuffled_X = X[permutation,:,:,:,:]
    shuffled_Y = Y[permutation,:]

    
    num_complete_minibatches = math.floor(m/mini_batch_size) 
    for k in range(0, num_complete_minibatches):
        mini_batch_X = shuffled_X[k * mini_batch_size : k * mini_batch_size + mini_batch_size,:,:,:,:]
        mini_batch_Y = shuffled_Y[k * mini_batch_size : k * mini_batch_size + mini_batch_size,:]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    

    if m % mini_batch_size != 0:
        mini_batch_X = shuffled_X[num_complete_minibatches * mini_batch_size : m,:,:,:,:]
        mini_batch_Y = shuffled_Y[num_complete_minibatches * mini_batch_size : m,:]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    return mini_batches


## Model


In [12]:
def model(X_train, Y_train, X_test, Y_test, learning_rate = 0.009,
          num_epochs = 100, minibatch_size = 64, print_cost = True):
   
    ops.reset_default_graph()                         
    tf.set_random_seed(1)                             
    seed = 3                                        
    (m, n_D0, n_R0, n_C0,n_Ch0) = X_train.shape            # shape of input image  
    n_y = Y_train.shape[1]                            #no of channels in output 
    costs = []                                        #list to store cost funtion
    shape=[n_D0,n_R0,n_C0]

    X, Y =placeholders(shape,n_Ch0, n_y)               #initializing placeholders
    
    parameters = initialize_parameters(n_Ch0)          #initializing placeholder

    Z3 = forward_prop(X, parameters)

    cost = com_cost(Z3, Y)                              #Computing cost

    optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(cost)  #optimizer to optimize and backprop

    init = tf.global_variables_initializer()
     

    with tf.Session() as sess:
  
        sess.run(init)
        
 
        for epoch in range(num_epochs): # function to create mini-bataches
            minibatch_cost = 0.
            num_minibatches = int(m / minibatch_size) 
            seed = seed + 1
            minibatches = random_mini_batches(X_train, Y_train, minibatch_size, seed)

            for minibatch in minibatches:


                (minibatch_X, minibatch_Y) = minibatch
                
                _ , temp_cost = sess.run([optimizer, cost], feed_dict={X: minibatch_X, Y: minibatch_Y})
           
                
                minibatch_cost += temp_cost / num_minibatches
                

   
            if print_cost == True and epoch % 5 == 0:
                print ("Cost after epoch %i: %f" % (epoch, minibatch_cost))
            if print_cost == True and epoch % 1 == 0:
                costs.append(minibatch_cost)
        
        

        plt.plot(np.squeeze(costs))
        plt.ylabel('cost')
        plt.xlabel('iterations (per tens)')
        plt.title("Learning rate =" + str(learning_rate))
        plt.show()


        predict_op = tf.argmax(Z3, 1)
        correct_prediction = tf.equal(predict_op, tf.argmax(Y, 1))
        

        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
        print(accuracy)
        train_accuracy = accuracy.eval({X: X_train, Y: Y_train})
        test_accuracy = accuracy.eval({X: X_test, Y: Y_test})
        print("Train Accuracy:", train_accuracy)
        print("Test Accuracy:", test_accuracy)
                
        return train_accuracy, test_accuracy, parameters

In [13]:
X_train = np.random.rand(500,30,64,64,3)*100
Y_train = np.random.rand(200,1)
Y_train = Y_train>=0.6
Y_train = Y_train*1
X_test  = np.random.rand(50,30,64,64)*100
Y_test = np.random.rand(50,1)
Y_test = Y_test>=0.6
Y_test = Y_test*1

In [ ]:
_, _, parameters = model(X_train, Y_train, X_test, Y_test)